In [ ]:
'''
VMP 2022-02-22: 
sanity check document. 
some papers are in paper_ref and paper_author but not in meta. 
because of the filters. 
here we check / show that the filters are reasonable. 
this file can be deleted again (only kept for convenience)
'''

In [1]:
import sys  
sys.path.insert(0, '/home/vicp')
from MAGsparkmasters import get_mag_with_cluster_connection
from MAGmasters import MicrosoftAcademicGraph
import os
from pyspark.sql import functions as F, Window
import pandas as pd
import matplotlib.pyplot as plt
import datetime, time
import numpy as np
from pyspark.sql.functions import avg
os.chdir('/home/vicp')
mag, spark = get_mag_with_cluster_connection(66699, 
                               memory_per_executor=16000)

['NAME STATE JOBID', 'cluster_new.job RUNNING 66699', 'cluster_new.job RUNNING 66698', 'jupyter-notebook RUNNING 66697', 'train-gan RUNNING 66694', 'train-gan RUNNING 66518', '']


In [2]:
spark

In [ ]:
# stress testing something
fos = 'psychology'
focusid = 2085043535 # this one was actually about replication (with replication in title) - did not have level 0 fos, but did have level 1 fos (data science)
focusid = 1960674264 # does not even have a fild of study and is in spanish
# generally seems that they are low quality papers

In [ ]:
# load files 
papers = mag.getSubset('Papers', ['PaperId', 'PaperTitle', 'DocType', 'Date']) # PaperId, DocType, Date, FamilyId. 
fos_papers = mag.getDataframe(f'{fos}_total') # PaperId
pfs = mag.getDataframe('PaperFieldsOfStudy') # couples PaperId and FieldOfStudyId
FoS = mag.getDataframe('FoS') # couples FieldOfStudyId and NormalizedName

In [ ]:
fos_papers.filter(fos_papers.PaperId == focusid).show(truncate=False)

In [ ]:
step1 = papers.filter(papers.PaperId == focusid)

In [ ]:
step1.show(truncate=False) # matches search term?

In [ ]:
step2 = step1.join(pfs, ['PaperId'], 'inner')

In [ ]:
step2.show(truncate=False)

In [ ]:
step3 = step2.join(FoS, ['FieldOfStudyId'], 'inner')

In [ ]:
step3.show(truncate=False)

In [ ]:
step4 = step3.filter(F.col('level') == 0)

In [ ]:
step4.show(truncate=False)

## check that there are only unique paper-ids in meta

In [1]:
import pandas as pd 

In [2]:
meta_clean = pd.read_csv("/home/vicp/data/2021-08-02/masters/psychology_paper_meta_clean.csv")

In [3]:
meta_clean.groupby('PaperId').size().reset_index(name = 'N').groupby('N').size().reset_index(name = 'frequency')

,N,frequency
0,1,4753942


# do we have some duplicate FamilyId in subset?
Could be that some of the papers are BOTH published as Journal & as Conference. 

In [10]:
def subset_data_check(fos): 
    papers = mag.getDataframe('Papers') # PaperId, DocType, Date, FamilyId. 
    fos_papers = mag.getDataframe(f'{fos}_papers') # FieldOfStudyID, PaperId
    
    # filter papers first - core data set. 
    sub = papers.filter((F.col('DocType') == 'Journal') | (F.col('DocType') == 'Conference')) \
        .filter((F.col('Date') >= datetime.date(2005, 1, 1)) & (F.col('Date') <= datetime.date(2021, 1, 1))) \
        .join(fos_papers, ['PaperId'], 'inner') \
        .select('PaperId', 'DocType', 'FamilyId', 'Date') \
        .distinct()
    print(f"{fos} dtypes: {sub.dtypes}")
    
    return sub

In [11]:
psych = subset_data_check('psychology')

psychology dtypes: [('PaperId', 'bigint'), ('DocType', 'string'), ('FamilyId', 'bigint'), ('Date', 'date')]


In [14]:
psych.select('FamilyId').distinct().count()

31397

In [16]:
psych.select('PaperId').distinct().count()

2498885

In [17]:
psych.distinct().count()

2498885

In [ ]:
# check up on this

In [18]:
psych.show(20)

+-------+----------+--------+----------+
|PaperId|   DocType|FamilyId|      Date|
+-------+----------+--------+----------+
|  42997|   Journal|    null|2007-01-01|
| 109356|   Journal|    null|2013-01-01|
| 261577|   Journal|    null|2007-01-01|
| 289171|   Journal|    null|2009-01-01|
| 310575|   Journal|    null|2005-01-01|
| 406435|   Journal|    null|2012-05-28|
| 540469|   Journal|    null|2010-06-27|
| 605810|Conference|    null|2013-01-01|
| 669189|   Journal|    null|2007-01-01|
| 777721|Conference|    null|2012-01-01|
| 957813|   Journal|    null|2013-10-01|
|1063622|   Journal|    null|2006-12-01|
|1118937|   Journal|    null|2007-04-10|
|1310691|   Journal|    null|2010-12-01|
|1530461|   Journal|    null|2005-11-01|
|1682525|Conference|    null|2010-07-08|
|1719704|   Journal|    null|2011-01-01|
|1749997|   Journal|    null|2011-10-01|
|1854942|   Journal|    null|2005-10-01|
|1865112|   Journal|    null|2012-07-31|
+-------+----------+--------+----------+
only showing top